# Segmentation Training Review

Image segmentation converts an input image to a mask where each pixel value represents an object class at that pixel.  This notebook provides summary informaiton for image segmentation algorithms.  .  


In [96]:
import sys, os
import json
import argparse
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
import ipywidgets as widgets
import plotly
import plotly.graph_objects as graph_obj
import plotly.figure_factory as ff

sys.path.insert(0, os.path.abspath('.'))
sys.path.insert(0, os.path.abspath('..'))
from utils.s3 import s3store, Connect
from utils.jsonutil import WriteDictJson, ReadDictJson
from segment.segmenttest import PrepareResults, PlotConfusion, UpdateConfusion, ClearOutput, PlotModels, UpdateModel

In [97]:
output = widgets.Output()

In [98]:
def parse_arguments():
    parser = argparse.ArgumentParser(description='Process arguments')
    parser.add_argument('-credentails', type=str, default='../creds.json', help='Credentials file.')

    parser.add_argument('-model_type', type=str,  default='segmentation')
    parser.add_argument('-test_results', type=str, default='test_results.json')   

    args = parser.parse_args("")
    return args

In [99]:
with output:
    args = parse_arguments()
    print('parse_arguments()={}'.format(args))

In [100]:
with output:
    s3, creds, s3def = Connect(args.credentails)
    print('Connect()={}'.format(s3def))

In [101]:
test_path = '{}/{}/{}'.format(s3def['sets']['test']['prefix'], args.model_type, args.test_results)
test_data = s3.GetDict(s3def['sets']['test']['bucket'], test_path)

In [102]:
test_names, overview, results, models  = PrepareResults(test_data)

In [103]:
confusion_display = graph_obj.FigureWidget()
if len(test_data) > 0:
    plot = PlotConfusion(test_data[0]['objects'], test_data[0]['results']['confusion'])
    confusion_display = graph_obj.FigureWidget(plot)

In [104]:
test_names, overview, results, model_dict = PrepareResults(test_data)
modelsplot = PlotModels(list(model_dict.values())[0])
modelsplot.update_layout(autosize=False, width=800, height=600)
models_display = graph_obj.FigureWidget(modelsplot)

In [105]:
models_select = widgets.Select(
    options=model_dict.keys(),
    description='Model Class:',
    disabled=False,
    rows=10,
    layout=widgets.Layout(width="30%"))

In [106]:
def ModelsSelect(change, plot, results):
    UpdateModel(plot, model_dict[change.new])

models_select.observe (lambda change:ModelsSelect(change, models_display, model_dict), names="value")

In [107]:
test_select = widgets.Select(
    options=test_names,
    description='Test:',
    disabled=False,
    rows=25,
    layout=widgets.Layout(width="40%"))

In [108]:
def Confusion(change, confusion, results):
    UpdateConfusion(confusion, results[change.new]['confusion'])

test_select.observe (lambda change:Confusion(change, confusion_display, results), names="value")

In [109]:
clear_output = widgets.Button(description='Clear Output')
clear_output.on_click(lambda b: ClearOutput(b, output=output))

## Segmentation Summary Table
This table displays the validation dataset test results for a specific model.  
- "test images" is the number of images in the validation data set
- "mean IoU" is the average of the per-class intersection over union defined in ["The PASCAL Visual Object Classes (VOC) Challenge" equation 4](https://homepages.inf.ed.ac.uk/ckiw/postscript/ijcv_voc09.pdf) 
- "inference time" is the average compute time per image for the CNN execution and argmax opration to produce a segmentation image.

In [110]:
display(pd.DataFrame(overview).T)

,model type,model class,test images,mean IoU,inference time,description
1: segment_deeplabv3_512x442_20211030_01,segmentation,deeplabv3,5000,0.709993,0.006181,
2: segment_deeplabv3_512x442_20211107_00,segmentation,deeplabv3,5000,0.790635,0.006192,
3: segment_deeplabv3_512x442_20211101_00,segmentation,deeplabv3,5000,0.782202,0.006293,
4: segment_deeplabv3_512x442_20211112_00,segmentation,deeplabv3,5000,0.803974,0.006058,
5: segment_deeplabv3_512x442_20211112_00,segmentation,deeplabv3,5000,0.809978,0.005556,
6: segment_deeplabv3_512x442_20211112_00,segmentation,deeplabv3,144,0.767238,0.0065,
7: segment_nas_512x442_20211114_01,segmentation,segmin,5000,0.73669,0.006011,Neural architecture search segmentation
8: segment_nas_512x442_20211101_00,segmentation,segmin,5000,0.727587,0.00609,Neural architecture search segmentation
9: segment_nas_512x442_20211029_00,segmentation,segmin,5000,0.700374,0.006111,Neural architecture search segmentation
10: segment_nas_512x442_20211028,segmentation,segmin,5000,0.624807,0.006104,Neural architecture search segmentation


In [111]:
## Model Confusion

In [112]:
display(widgets.HBox([test_select, confusion_display]))


# Initialize widgits after callbacks are defined
if len(test_names) > 0:
    test_select.options = test_names
    test_select.value = test_names[0]

## Segmentation Models Plot
The "Segmentaiton Models Plot" plots the per class and mean IoU through the training sequence of a sepcific model type and class.  Comparing these plot, I evalute the potential improvement of a specific type of model.  For example, if the IoU continue to improve with training, I'll continue to train the model.  If the per-class IoU converges with additonal training, it may not be neccessary to manipulate the class balance.  Does one model convergence more quickly or to a higher value than another?

In [113]:
display(widgets.HBox([models_select, models_display]))

## Runtime Output Log

In [114]:
display(widgets.VBox([clear_output, output]))